# Spaceship Titanic Dataset with XGBoost


# Import the packages


In [1]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from IPython.display import display

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import accuracy_score


from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import optuna

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings("ignore")

c:\Users\Vasya\.pyenv\pyenv-win\versions\3.11.2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Vasya\AppData\Local\Temp\ipykernel_6328\2805310176.py:24: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


In [2]:
# PATH = "/kaggle/input/spaceship-titanic/"
PATH = ""

In [3]:
target = "Transported"

# Data preprocessing


## Clean data


In [4]:
def bool_to_int(x):
    if str(x) == "True":
        return 1
    elif str(x) == "False":
        return 0
    else:
        return x


def clean(df):
    df["Transported"] = df["Transported"].astype("bool")

    return df

## Encode


In [5]:
def encode(df):
    features_nom = df.select_dtypes(exclude=["number", "bool"])

    for name in features_nom:
        df[name] = df[name].astype("category")

        if "None" not in df[name].cat.categories:
            df[name] = df[name].cat.add_categories("None")
    return df

## Load Data


In [6]:
def load_data(PATH, with_test=True):
    # Read data
    data_dir = Path(PATH)
    df = pd.read_csv(data_dir / "train.csv")
    id_train = df.index
    if with_test:
        # Merge the splits so we can process them together
        df_test = pd.read_csv(data_dir / "test.csv")
        id_test = df_test.index + len(id_train)
        df = pd.concat([df, df_test], ignore_index=True)
    # Cleaning
    df = clean(df)
    df = encode(df)
    # Reform splits
    df_train = df.loc[id_train, :]
    if with_test:
        df_test = df.loc[id_test, :]
        return df_train, df_test
    return df_train

## Impute values


In [7]:
def impute_cryosleep_expences(df):
    df_imputed = df.copy()
    expences = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
    df_imputed.loc[df_imputed[expences].sum(axis=1) == 0, "CryoSleep"] = True
    df_imputed.loc[df_imputed[expences].sum(axis=1) != 0, "CryoSleep"] = False
    df_imputed.loc[df_imputed["CryoSleep"] == True, expences] = 0

    return df_imputed


def label_encode_keeping_nulls(df):
    df_encoded = df.copy()
    for col in df.select_dtypes("category").columns:
        indx = df.loc[~df[col].isna(), col].index
        df_encoded[col] = df[col].astype("object")
        df_encoded.loc[indx, col] = df[col].cat.codes[indx]
    return df_encoded


def impute_using_group(df):
    df_groups = df.copy()

    df_groups["Group"] = df_groups.PassengerId.str[:4]
    df_groups = df_groups[df_groups.groupby(["Group"])["Group"].transform("count") > 1]
    df_encoded = label_encode_keeping_nulls(df_groups)

    features_impute = ["Cabin", "Destination", "HomePlanet"]
    for feature in features_impute:
        features_knn = ["Group", feature]
        df_knn_encoded = df_encoded[features_knn]
        knn_imputer = KNNImputer(n_neighbors=1, weights="distance")
        df_knn_imputed = pd.DataFrame(
            knn_imputer.fit_transform(df_knn_encoded),
            columns=df_knn_encoded.columns,
            index=df_knn_encoded.index,
        )
        df_knn_imputed = df_knn_imputed.round(0)

        for name in features_knn[1:]:
            cat_items = dict(enumerate(df[name].cat.categories))
            df_knn_imputed[name] = df_knn_imputed[name].replace(cat_items)
            idx = df_groups[name].isna().index
            df.loc[idx, name] = df_knn_imputed.loc[idx, name]

    return df


def knn_impute(df, df_train=None):
    df_encoded = label_encode_keeping_nulls(df)

    std_scaler = StandardScaler()
    df_scaled = df_encoded.copy()
    df_scaled[df_scaled.columns] = std_scaler.fit_transform(df_encoded)

    knn_imputer = KNNImputer(weights="distance")
    knn_cat_features = ["VIP", "CryoSleep", "HomePlanet", "Destination"]
    if df_train is not None:
        knn_imputer.fit(df_scaled.loc[df_train.index])
    else:
        knn_imputer.fit(df_scaled)
    df_scaled[df_scaled.columns] = knn_imputer.transform(df_scaled)

    df_encoded[df_encoded.columns] = std_scaler.inverse_transform(df_scaled)

    df_encoded = df_encoded.round(0)
    for name in knn_cat_features:
        df_encoded[name] = (
            df_encoded[name].astype("category").cat.add_categories("None")
        )
        new_categories = df[name].astype("category").cat.categories
        df[name] = (
            df_encoded[name].astype("category").cat.rename_categories(new_categories)
        )

    num_features = df.select_dtypes("number").columns
    df[num_features] = df_encoded[num_features]

    return df

In [8]:
def impute(df_train, df_test=None):
    df = df_train.copy()
    if df_test is not None:
        df = pd.concat([df, df_test])
    y = df.pop(target)

    df = impute_cryosleep_expences(df)
    df = impute_using_group(df)

    feature_bfills = ["Cabin"]
    df[feature_bfills] = df[feature_bfills].fillna(method="bfill")

    if df_test is not None:
        df = knn_impute(df, df_train)
    else:
        df = knn_impute(df)

    for name in df.select_dtypes("number"):
        df[name] = df[name].fillna(0).astype("int")
    for name in df.select_dtypes(exclude=["number"]):
        if df[name].isna().sum() != 0:
            df[name] = df[name].fillna("None")
        else:
            df[name] = df[name].cat.remove_categories("None")

    df = pd.concat([df, y], axis=1)
    if df_test is not None:
        return df.loc[df_train.index, :], df.loc[df_test.index, :]

    return df

# Features engineering


In [9]:
def mathematical_transforms(df):
    X = pd.DataFrame()
    X["RS_FC_SM"] = df[["RoomService", "FoodCourt", "ShoppingMall"]].sum(axis=1)
    X["VR_SPA"] = df[["Spa", "VRDeck"]].sum(axis=1)
    X["Overall_spent"] = X["RS_FC_SM"] + X["VR_SPA"]
    X["Ratio_VR_SPA"] = X["VR_SPA"] / X["Overall_spent"]
    X["Ratio_RFS"] = X["RS_FC_SM"] / X["Overall_spent"]

    X["Sleep_Side"] = df["CryoSleep"].cat.codes * df["Side"].cat.codes

    return X


def interactions(df):
    X = pd.DataFrame()

    dummies_sleep = pd.get_dummies(df.CryoSleep, prefix="CryoSleep_VR_SPA")
    sleep_VR_SPA = dummies_sleep.mul(df.VR_SPA, axis=0)

    dummies_sleep = pd.get_dummies(df.CryoSleep, prefix="CryoSleep_RFM")
    sleep_RFS = dummies_sleep.mul(df.RS_FC_SM, axis=0)

    X = pd.concat([sleep_VR_SPA, sleep_RFS], axis=1)

    return X


def break_down(df):
    X = pd.DataFrame()

    X[["Group", "Id"]] = df.PassengerId.str.split("_", expand=True)
    X["Group"] = X.Group.astype("int")
    X.pop("Id")

    X[["Deck", "Cabin_num", "Side"]] = df.Cabin.str.split("/", expand=True)
    X[["Deck", "Side"]] = X[["Deck", "Side"]].fillna("None")
    X.pop("Cabin_num")

    X[["First Name", "Surname"]] = df.Name.str.split(" ", expand=True).fillna("None")
    X.pop("First Name")
    for name in ["Deck", "Side", "Surname"]:
        X[name] = X[name].astype("category")

        if "None" not in X[name].cat.categories:
            X[name] = X[name].cat.add_categories("None")

    return X


def group_transform(df):
    X = pd.DataFrame()

    X["Diff_VR_SPA"] = df["VR_SPA"] - df.groupby(by=["CryoSleep"])["VR_SPA"].transform(
        "median"
    )
    X["Diff_RS_FC_SM"] = df["RS_FC_SM"] - df.groupby(by=["CryoSleep"])[
        "RS_FC_SM"
    ].transform("median")

    X["in_group"] = (df.groupby(["Group"])["Group"].transform("count") > 1).astype(
        "int"
    )

    return X

In [10]:
def label_encode(df):
    X = df.copy()
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    return X


def create_dummies(df, features):
    X = pd.DataFrame()
    for name in features:
        X = pd.concat([X, pd.get_dummies(df[name], prefix=name).astype("int")], axis=1)
    return X

## K-Means clustering


In [11]:
def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    X_new = pd.DataFrame()
    X_new["Clusters"] = kmeans.fit_predict(X_scaled)
    return X_new

# Create final feature set


In [12]:
def create_features(df, df_test=None):
    X = df.copy()
    y = X.pop(target)

    # Combine splits if test data is given
    #
    # If we're creating features for test set predictions, we should
    # use all the data we have available. After creating our features,
    # we'll recreate the splits.
    if df_test is not None:
        X_test = df_test.copy()
        if target in X_test.columns:
            X_test.pop(target)
        X = pd.concat([X, X_test])

    # Transformations
    X = X.join(break_down(X))
    X = X.join(mathematical_transforms(X))
    X = X.join(interactions(X))
    X = X.join(group_transform(X))

    X = label_encode(X)

    # Clustering
    cluster_features = [
        "CryoSleep",
        "Side",
        "Deck",
        "FoodCourt",
        "ShoppingMall",
        "Spa",
        "VRDeck",
    ]

    X = X.join(cluster_labels(X, cluster_features, n_clusters=10))

    # Create dummies
    features = ["Deck", "HomePlanet", "Destination", "Side"]
    X = X.join(create_dummies(X, features))
    X.drop(features, axis=1, inplace=True)

    # Drop categorical features that can be hardly match between train and test sets
    X.drop(["PassengerId", "Name", "Cabin", "Group", "Surname"], axis=1, inplace=True)

    # Reform splits
    if df_test is not None:
        X_test = X.loc[df_test.index, :]
        X.drop(df_test.index, inplace=True)

    if df_test is not None:
        return X, X_test
    else:
        return X

# Cross validation


In [13]:
def score_dataset(X, y, model=XGBClassifier(), features=None):
    score = []
    skf = StratifiedKFold(random_state=0, shuffle=True)
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train, X_test = impute(X_train, X_test)
        X_train, X_test = create_features(X_train, X_test)

        if features is not None:
            X_train, X_test = X_train[features], X_test[features]

        model_local = clone(model)
        model_local.fit(X_train, y_train)

        score.append(accuracy_score(model_local.predict(X_test), y_test))

    return score

# Hyperparameter Tuning


## XGBoost


In [14]:
# optuna.logging.set_verbosity(optuna.logging.WARNING)


# def objective(trial):
#     df_train = load_data(PATH, with_test=False)
#     y_train = df_train.loc[:, target].astype("bool")

#     xgb_params = dict(
#         max_depth=trial.suggest_int("max_depth", 2, 10),
#         learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
#         n_estimators=trial.suggest_int("n_estimators", 50, 5000),
#         min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
#         colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         subsample=trial.suggest_float("subsample", 0.2, 1.0),
#         reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e1, log=True),
#         reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e1, log=True),
#     )
#     xgb = XGBClassifier(**xgb_params)

#     return np.mean(score_dataset(df_train, y_train.astype('int'), xgb))


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)
# xgb_params = study.best_params

In [15]:
# print("The best parameters")
# xgb_params

In [16]:
df_train = load_data(PATH, with_test=False)

y_train = df_train.loc[:, target].astype("bool")

xgb_params = {
    "max_depth": 6,
    "learning_rate": 0.0011231556765223495,
    "n_estimators": 4968,
    "min_child_weight": 1,
    "colsample_bytree": 0.6486673116722138,
    "subsample": 0.5656591278713615,
    "reg_alpha": 0.03102176365621891,
    "reg_lambda": 0.06932993934004869,
}

xgb = XGBClassifier(**xgb_params)

print(np.mean(score_dataset(df_train, y_train, xgb)))

0.810075893781792


## Catboost


In [17]:
# def objective(trial):
#     params = dict(
#         iterations=trial.suggest_int("iterations", 50, 1000),
#         learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
#         depth=trial.suggest_int("depth", 4, 10),
#         l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
#         bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
#         random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
#         bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
#         od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
#         od_wait=trial.suggest_int("od_wait", 10, 50),
#     )

#     cat_boost = CatBoostClassifier(**params, verbose=0, random_seed=0)

#     return np.mean(score_dataset(df_train, y_train.astype("int"), cat_boost))


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100)
# cat_boost_params = study.best_params

[I 2023-08-15 17:12:10,305] A new study created in memory with name: no-name-d5017cfc-70af-498f-b818-d3cb9e370f0c
[I 2023-08-15 17:13:40,482] Trial 0 finished with value: 0.7951208682423334 and parameters: {'iterations': 716, 'learning_rate': 0.015859087108804622, 'depth': 9, 'l2_leaf_reg': 0.032970857189770844, 'bootstrap_type': 'Bayesian', 'random_strength': 8.588970480929878e-06, 'bagging_temperature': 3.1702530767628234, 'od_type': 'Iter', 'od_wait': 43}. Best is trial 0 with value: 0.7951208682423334.
[I 2023-08-15 17:14:10,445] Trial 1 finished with value: 0.7799374136029132 and parameters: {'iterations': 732, 'learning_rate': 0.0011916217168881742, 'depth': 4, 'l2_leaf_reg': 3.872137959692176e-08, 'bootstrap_type': 'Bayesian', 'random_strength': 1.457107446162148e-07, 'bagging_temperature': 9.82952420707752, 'od_type': 'IncToDec', 'od_wait': 30}. Best is trial 0 with value: 0.7951208682423334.
[I 2023-08-15 17:14:44,414] Trial 2 finished with value: 0.7943168004573876 and parame

In [18]:
# print("The best parameters")
# cat_boost_params

The best parameters


{'iterations': 604,
 'learning_rate': 0.008574858135413305,
 'depth': 6,
 'l2_leaf_reg': 0.024986714637090388,
 'bootstrap_type': 'Bayesian',
 'random_strength': 3.700277878995026e-05,
 'bagging_temperature': 0.027325609307599474,
 'od_type': 'Iter',
 'od_wait': 39}

In [23]:
df_train = load_data(PATH, with_test=False)

y_train = df_train.loc[:, target].astype("int")

cat_boost_params = {
    "iterations": 604,
    "learning_rate": 0.008574858135413305,
    "depth": 6,
    "l2_leaf_reg": 0.024986714637090388,
    "bootstrap_type": "Bayesian",
    "random_strength": 3.700277878995026e-05,
    "bagging_temperature": 0.027325609307599474,
    "od_type": "Iter",
    "od_wait": 39,
}

cat_boost = CatBoostClassifier(**cat_boost_params, verbose=0)

print(np.mean(score_dataset(df_train, y_train, cat_boost)))

0.8119171567326697


# Train model and create submission


In [26]:
df_train, df_test = load_data(PATH)
target = "Transported"
X_train, X_test = impute(df_train, df_test)
X_train, X_test = create_features(X_train, X_test)
y_train = df_train.loc[:, target].astype("int").to_numpy()

cat_boost = CatBoostClassifier(**cat_boost_params, verbose=0)
cat_boost.fit(X_train, y_train)
predictions = cat_boost.predict(X_test).astype("bool")

output = pd.DataFrame(
    {"PassengerId": df_test.PassengerId, "Transported": predictions.squeeze()}
)

output.to_csv("submission.csv", index=False)